In [ ]:
from exp.features import create_train_features
from exp.run import run_experiment
from exp.mappings import alg_map
from exp.train import train_model
import pandas as pd
import json
import numpy as np

### Create Training Features

In [ ]:
X_save = "X_tr.csv"
y_save = "y_tr.csv"
X_save_scaled = "X_tr_scaled.csv"
scale_params_pickle = "scale_params.pickle"
other_params_json = "other.json"
tr_scaler = None
classic_sta_lta5_mean_fill = None
classic_sta_lta7_mean_fill = None

if not (os.path.exists(X_save_scaled) and os.path.exists(y_save)):
    if os.path.exists(X_save) and os.path.exists(y_save):
        X_tr = pd.read_csv(X_save, index_col=0)
        y_tr = pd.read_csv(y_save, index_col=0)

        scale_params_pickle_on = open(scale_params_pickle, "rb")
        tr_scaler = pickle.load(scale_params_pickle_on)
        scale_params_pickle_on.close()
        
        X_train_scaled = pd.DataFrame(tr_scaler.transform(X_tr), columns=X_tr.columns)
        X_train_scaled.to_csv(X_save_scaled)
    else:
        X_tr, X_train_scaled, y_tr, tr_scaler, classic_sta_lta5_mean_fill, classic_sta_lta7_mean_fill  = create_train_features(r'C:\Users\arvin\dev\lanl\train.csv')
        X_tr.to_csv(X_save)
        y_tr.to_csv(y_save)
        X_train_scaled.to_csv(X_save_scaled)

        scale_params_pickle_on = open(scale_params_pickle, "wb")
        pickle.dump(tr_scaler, scale_params_pickle_on)
        scale_params_pickle_on.close()

        with open(other_params_json, 'w') as fp:
            json.dump({"classic_sta_lta5_mean_fill": classic_sta_lta5_mean_fill,
                       "classic_sta_lta7_mean_fill": classic_sta_lta7_mean_fill}, fp)
else:
    X_train_scaled = pd.read_csv(X_save_scaled, index_col=0)
    y_tr = pd.read_csv(y_save, index_col=0)

### hyper-parameter experiments

In [ ]:
"""
Example of Cartesian Product of Hyper-parameters for Linear Regression

"lr": {"fit_intercept": [False, True], "normalize": [False, True]}

Cartesian Product: {fit_intercept} x {normalize}

Hyper-parameter choices:
"fit_intercept": False, "normalize": False
"fit_intercept": True, "normalize": False
"fit_intercept": False, "normalize": True
"fit_intercept": True, "normalize": True
"""

params={"lr": {"fit_intercept": [False, True], "normalize": [False, True]},
       "ridge": {"alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                 "fit_intercept": [False, True], "normalize": [False, True]},
       "lasso": {"alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                 "fit_intercept": [False, True], "normalize": [False, True],
                 "positive": [False, False, False, False, False, True],
                 "selection": ["cyclic", "cyclic", "cyclic", "cyclic", "cyclic", "random"]},
       "mtlasso": {"alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                   "fit_intercept": [False, True], "normalize": [False, True],
                   "selection": ["cyclic", "cyclic", "cyclic", "cyclic", "cyclic", "random"]},
       "elastic": {"alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                   "fit_intercept": [False, True], "normalize": [False, True], 
                   "positive": [False, False, False, False, False, True],
                   "l1_ratio": [.01, .99, .2, .4, .6, .8], 
                   "selection": ["cyclic", "cyclic", "cyclic", "cyclic", "cyclic", "random"]},
       "lars": {"fit_intercept": [False, True], "normalize": [False, True],
                "fit_path": [False], "n_nonzero_coefs": [10, 100, 500, 1000, 10000, np.inf]},
       "llars": {"alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                 "fit_intercept": [False, True], "normalize": [False, True],
                "fit_path": [False], "positive": [False, False, False, False, False, True]},
       "omp": {"fit_intercept": [False, True], "normalize": [False, True],
               "n_nonzero_coefs": [10, 100, None, None, None]},
       "sgdreg": {"loss": ["squared_loss", "squared_loss", "squared_loss", "huber", "epsilon_insensitive",
                           "squared_epsilon_insensitive"],
                  "penalty": ["none", "l2", "l1", "elasticnet"], 
                  "alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                  "l1_ratio": [.01, .99, .2, .4, .6, .8], "fit_intercept": [False, True],
                  "learning_rate": ["constant", "optimal", "optimal", "optimal", "invscaling", "adaptive"],
                  "eta0": [1.0, 10.0, .1, .01, .001, .0001],
                  "early_stopping": [False, False, False, False, True]},
       "pareg": {"C": [.001, .01, .1, 1.0, 1.0, 1.0, 10.0, 100.0],
                 "loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
                 "epsilon": [.01, .05, .1, .1, .1, .5],
                 "early_stopping": [False, False, False, False, True]},
        # "tsreg": {"fit_intercept": [False, True]},
        "hreg": {"epsilon": [1.1, 1.2, 1.35, 1.35, 1.35, 1.35, 1.5, 1.6, 1.8, 2.0, 2.5],
                 "alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                 "fit_intercept": [False, True]},
        "kreg": {"alpha": [.000001, .00001, .0001, .001, .01, .1, 1.0, 10, 100],
                 "kernel": ["linear", "linear", "poly", "rbf", "sigmoid"],
                 "gamma": [None, None, None, None, .001, .0001, .01, .1]}}

### Run Experiment

In [ ]:
num_searches=20
n_fold=10
save_results= "exp.csv"

In [ ]:
for alg in params.keys():
    print(alg)
    score_df = run_experiment(X=X_train_scaled, Y=y_tr, n_fold=n_fold, alg=alg, alg_params=params[alg], search_type="random", num_searches=num_searches, save_results=save_results)

### Display models ranked by CV scores

In [ ]:
score_df = score_df.sort_values(by="score", axis=0)
display(score_df)

### Load results from CSV File and re-produce models ranked by CV scores

In [ ]:
score_df = pd.read_csv(save_results)
score_df = score_df.sort_values(by="score", axis=0)

In [ ]:
display(score_df)

### Load best model from CSV File

In [ ]:
# retrieve top scoring row
best = score_df.iloc[0]
display(best)

# retrieve model parameters from pandas row
alg = best["alg"]
params_json = best["params_json"]
print("alg: {}".format(alg))
print("params_json: {}".format(params_json))

# retrieve relevant values
alg_cls = alg_map[alg]
params = json.loads(params_json)

# initialize model
model = alg_cls(**params)

# train algorithm
train_model(X=X_train_scaled, Y=y_tr, n_fold=n_fold, model=model)